In [ ]:
#Парсер постов и их хараткеристик ВК (за последние 5 лет)
import configparser
import vk_api
from vk_api.exceptions import ApiError
import pandas as pd
from datetime import datetime, timedelta

def main():
    config = configparser.ConfigParser()
    config.read('config.ini') #файл для выгрузки токена API VK
    token = config.get('VK', 'TOKEN')

    vk_session = vk_api.VkApi(token=token)
    vk = vk_session.get_api()

    keywords = ["экологический активизм", "экоактивизм", "экология", "эко"]
    rows = []
    closed_groups_count = 0
    five_years_ago = datetime.now() - timedelta(days=5 * 365)

    for keyword in keywords:
        try:
            groups = vk.groups.search(q=keyword, count=10, type='group', filter='publics')
        except ApiError as error:
            print(f"Ошибка поиска групп по ключевому слову {keyword}: {error}")
            continue
        
        for group in groups['items']:
            if group['is_closed']:
                closed_groups_count += 1
                continue
            
            group_id = group['id']
            try:
                group_info = vk.groups.getById(group_ids=group_id, fields='members_count')[0]
                group_name = group_info['name']
                group_members_count = group_info.get('members_count', 'недоступно')
                
                posts = vk.wall.get(owner_id=f'-{group_id}', count=100)
                for post in posts['items']:
                    post_date = datetime.fromtimestamp(post['date'])
                    if post_date < five_years_ago:
                        continue

                    rows.append({
                        'Название группы': group_name,
                        'ID группы': group_id,
                        'Количество участников': group_members_count,
                        'ID поста': post['id'],
                        'Дата поста': post_date.strftime('%Y-%m-%d %H:%M:%S'),
                        'Текст поста': post['text'],
                        'Количество лайков': post.get('likes', {}).get('count', 0),
                        'Количество комментариев': post.get('comments', {}).get('count', 0),
                        'Количество репостов': post.get('reposts', {}).get('count', 0),
                        'Количество просмотров': post.get('views', {}).get('count', 0)
                    })
            except ApiError as e:
                print(f"Ошибка при получении информации о группе или постах из группы ID {group_id}: {e}")

    df = pd.DataFrame(rows)
    df.to_csv('ypur file name here.csv', index=False, encoding='utf_8_sig')
    print(f"Количество закрытых групп, с которых не удалось добыть данные: {closed_groups_count}")
    print(f"Сохранено {len(rows)} записей.")
    return df

if __name__ == '__main__':
    df = main()